In [1]:
import sys
sys.path.append('/home1/04287/mxhf/HETDEX_API/')

In [2]:
%matplotlib inline
import sys
import os
import os.path
import subprocess
import numpy as np

import tables as tb
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from astropy.visualization import ZScaleInterval
from astropy.table import Table

from hetdex_api.survey import Survey

In [3]:
survey = Survey('hdr1')

In [4]:
s = Table(survey.hdfile.root.Survey[:])

In [5]:
s = Table([s["datevobs"]])

In [6]:
N = len(s)

In [13]:
s.write("/work/04287/mxhf/maverick/hetdex/shotlist_hdr1.txt", format="ascii")

In [13]:
from IPython.parallel import Client
c = Client()
lv = c.load_balanced_view()   # this object represents the engines (workers)
print("Using IPython.parallel.")

Waiting for connection file: ~/.ipython/profile_default/security/ipcontroller-client.json
Using IPython.parallel.


In [ ]:
#build noise model
def run(shotid, DRYRUN=True):
    import os
    import tables as tb
    from astropy.table import Table
    oufilename = "/work/04287/mxhf/maverick/hetdex/cubes{shotid}.images.txt".format(shotid=shotid)
    if os.path.exists(oufilename):
        print("{} exists already.".format(oufilename))
        return 0
    
    print("Processing shot {}.".format(shotid))
    fileh = tb.open_file('/work/03946/hetdex/hdr1/reduction/data/{shotid}.h5'.format(shotid=shotid))
    l = Table( fileh.root.Data.Images[:] )
    l2 = Table( [l["contid"], l["ifuid"], l["ifuslot"], l["specid"]] )
    l2.write(oufilename, format="ascii")
    
    return 0

tasks = []

for shotid in s["datevobs"]:
    tasks.append(lv.apply(run, shotid))

result = [task.get() for task in tasks]  # blocks until all results are back

for r in result:
    print(r)

In [7]:
# non-parallel impleentation
for i, shotid in enumerate(s["datevobs"]):
    oufilename = "/work/04287/mxhf/maverick/hetdex/cubes{shotid}.images.txt".format(shotid=shotid)
    if os.path.exists(oufilename):
        #print("{} exists already.".format(oufilename))
        continue
    
    print("Processing shot {} which is {} out of {}.".format(shotid,i,N))
    fileh = tb.open_file('/work/03946/hetdex/hdr1/reduction/data/{shotid}.h5'.format(shotid=shotid))
    l = Table( fileh.root.Data.Images[:] )
    #l.write("{shotid}.images.txt".format(shotid=shotid), format="")
    l2 = Table( [l["contid"], l["ifuid"], l["ifuslot"], l["specid"]] )
    l2.write(oufilename, format="ascii")
    #break

Processing shot 20181004v007 which is 1433 out of 1714.
Processing shot 20181214v011 which is 1666 out of 1714.


In [18]:
!python create_master_table.py 20190208v034

Processing shot 20190208v034 .
Closing remaining open files:/work/03946/hetdex/hdr1/reduction/data/20190208v034.h5...done


In [20]:
!python ~/hetdex/hetdex-deep/create_master_table.py 20190118v010

Processing shot 20190118v010 .
Closing remaining open files:/work/03946/hetdex/hdr1/reduction/data/20190118v010.h5...done
